# Recreation of Figure 2 
## Hyperbolic map of the legal-entities network 
Step 1: Load the nodes coordinates

In [1]:
import re

# The Legal-entities network with the nodes coordinates 
f_name = "../Datasets/EdgeList_Legal_Entities.inf_coord"

header_map = {
    "Vertex": "id",
    "Inf.Kappa": "kappa",
    "Inf.Theta": "theta_rad",
    "Inf.Hyp.Rad.": "r"
}

# The final dataset
dataset = {}
# The output header variables
header = []
# The network nodes 
nodes = []
# The embedding parameters
parameters = {}

# Read the Mercator output file line by line
with open(f_name, "r") as fp:
    for l in fp.readlines():
        # Line cleanup step
        line = re.sub(' +', ' ', l.strip())
        # Process Comment line
        if line.startswith("#"):
            # Extract parameters
            if " - " in line:
                tmp = line.replace("# - ", "").split(": ")
                parameters[tmp[0]] = float(tmp[1])
            # Extract data header
            elif line.startswith("# Vertex"):
                tmp = line.replace("# ", "").split(" ")
                for e in tmp:
                    header.append(header_map[e])
        else: # Process nodes data
            tmp = line.split(" ")
            t = {}
            for k, v in zip(header, tmp):
                t[k] = v
            nodes.append(t)

dataset["parameters"] = parameters
dataset["nodes"] = nodes

print("# of nodes: {}".format(len(dataset["nodes"])))

# of nodes: 1215


Step 2: Load the links between the nodes in the network 

In [2]:
import re

links = []

# Load the Legal-entities edge list 
with open("../Datasets/EdgeList_Legal_Entities.txt", "r") as fp:
    for l in fp.readlines():
        line = re.sub(' +', ' ',l).strip().split(" ")
        links.append({
            "source": line[0],
            "target": line[1]
        })
dataset["links"] = links
print("# of links: {}".format(len(dataset["links"])))

# of links: 5920


Step 3: Calculate nodes degree

In [3]:
# Import required libraries
import pandas as pd
import networkx as nx
import operator

from_n = []
to_n = []

for l in dataset["links"]:
    from_n.append(l["source"])
    to_n.append(l["target"])

# Build a dataframe with all connections
df = pd.DataFrame({'from':from_n, 'to':to_n})
 
# Build the graph
G=nx.from_pandas_edgelist(df, 'from', 'to')

degrees = dict(G.degree())
sorted_degrees = sorted(degrees.items(), key=operator.itemgetter(1), reverse=True)
print("# of sorted nodes by degree: {}".format(len(sorted_degrees)))

# of sorted nodes by degree: 1215


Step 4: Import required metadata

In [4]:
# Import node type colors (pallets)
from metadata.pallets import *
dataset["country_pal"] = country_pal
dataset["category_pal"] = category_pal
print("country_pal: {}\ncategory_pal: {}".format(len(country_pal),len(category_pal)))

# Import nodes location and type
from metadata.Location_Type import *
print("LocAndType:", len(LocAndType))

country_pal: 56
category_pal: 25
LocAndType: 1647


Step 5: Annotate network nodes with metadata

In [5]:
import math

for n in dataset["nodes"]:
    n["name"] = n["id"]
    n["r"] = float(n["r"])
    n["theta"] = float(n["theta_rad"]) * (180 / math.pi)
    n["degree"] = degrees[n["id"]]
    try:
        n["Category"] = LocAndType[n["id"]]["category"]
        n["Country"] = LocAndType[n["id"]]["Country"]
    except:
        print(n["id"])
    
print("# of annotated nodes: {}".format(len(dataset["nodes"])))

# of annotated nodes: 1215


Step 6: Save the annotated nodes to a json file

In [6]:
import json

with open(f_name + ".json", "w") as fp:
    json.dump(dataset, fp, indent=2)